In [309]:
import numpy as np
import pandas
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, power_transform

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

### Define data directory and file name to load

In [4]:
data_directory='../data/raw'
file_name='Tuesday-WorkingHours.pcap_ISCX.csv'

data_file=os.path.join(data_directory,file_name)

Tried the package loader from github clean code

In [5]:
# from anomalyzer.config import data_file

### Load the data frame

In [6]:
df=pandas.read_csv(data_file,skipinitialspace=True,low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445909 entries, 0 to 445908
Data columns (total 79 columns):
Destination Port               445909 non-null int64
Flow Duration                  445909 non-null int64
Total Fwd Packets              445909 non-null int64
Total Backward Packets         445909 non-null int64
Total Length of Fwd Packets    445909 non-null int64
Total Length of Bwd Packets    445909 non-null int64
Fwd Packet Length Max          445909 non-null int64
Fwd Packet Length Min          445909 non-null int64
Fwd Packet Length Mean         445909 non-null float64
Fwd Packet Length Std          445909 non-null float64
Bwd Packet Length Max          445909 non-null int64
Bwd Packet Length Min          445909 non-null int64
Bwd Packet Length Mean         445909 non-null float64
Bwd Packet Length Std          445909 non-null float64
Flow Bytes/s                   445708 non-null object
Flow Packets/s                 445909 non-null object
Flow IAT Mean                  

Some of the features have the character '/' in them, which might be problematic. Therefore, such features are renamed to be on the safe side.

In [7]:
df.rename(columns={'Flow Bytes/s':'Flow Bytes per sec',
                   'Flow Packets/s':'Flow Packets per sec',
                   'Fwd Packets/s':'Fwd Packets per sec',
                   'Bwd Packets/s':'Bwd Packets per sec',
                   'Fwd Avg Bytes/Bulk':'Fwd Avg Bytes per Bulk',
                   'Fwd Avg Packets/Bulk':'Fwd Avg Packets per Bulk',
                   'Bwd Avg Bytes/Bulk':'Bwd Avg Bytes per Bulk',
                   'Bwd Avg Packets/Bulk':'Bwd Avg Packets per Bulk',
                   'Down/Up Ratio':'Down Up Ratio'
                  },inplace=True)

We also quickly check the distibution of attacks in the data set.

In [327]:
df['Label'].value_counts(normalize=True)

BENIGN         0.968973
FTP-Patator    0.017802
SSH-Patator    0.013225
Name: Label, dtype: float64

It looks like 96.90% of the row in our dataset are BENIGN communications and the remaining 3.10% are attacks on the network.

## Data Cleaning

### Remove NaN values from the data frame

In [8]:
clean_df=df.dropna()
clean_df.dtypes.value_counts()

int64      54
float64    22
object      3
dtype: int64

In [9]:
clean_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,88,640,7,4,440,358,220,0,62.857143,107.349008,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,88,900,9,4,600,2944,300,0,66.666667,132.287566,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,88,1205,7,4,2776,2830,1388,0,396.571429,677.274651,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,88,511,7,4,452,370,226,0,64.571429,110.276708,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,88,773,9,4,612,2944,306,0,68.000000,134.933317,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [10]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445708 entries, 0 to 445908
Data columns (total 79 columns):
Destination Port               445708 non-null int64
Flow Duration                  445708 non-null int64
Total Fwd Packets              445708 non-null int64
Total Backward Packets         445708 non-null int64
Total Length of Fwd Packets    445708 non-null int64
Total Length of Bwd Packets    445708 non-null int64
Fwd Packet Length Max          445708 non-null int64
Fwd Packet Length Min          445708 non-null int64
Fwd Packet Length Mean         445708 non-null float64
Fwd Packet Length Std          445708 non-null float64
Bwd Packet Length Max          445708 non-null int64
Bwd Packet Length Min          445708 non-null int64
Bwd Packet Length Mean         445708 non-null float64
Bwd Packet Length Std          445708 non-null float64
Flow Bytes per sec             445708 non-null object
Flow Packets per sec           445708 non-null object
Flow IAT Mean                  

There are still two 'object' type features in the dataset: Flow Bytes per sec and Flow Packets per sec

### We now need to remove 'Infinity' values typed as string from the data.

In [11]:
index_to_drop=clean_df.index[(clean_df['Flow Packets per sec']=='Infinity')|(clean_df['Flow Bytes per sec']=='Infinity')]
print(index_to_drop)

Int64Index([   167,    172,   2439,   4058,  15297,  22891,  28757,  33942,
             35342,  38478,  41582,  44806,  45262,  48751,  61676,  63183,
             67315,  88127,  93289,  93595,  93975, 111843, 118883, 120246,
            121444, 121536, 129524, 136265, 137752, 137844, 138921, 147048,
            150093, 153792, 154300, 154805, 157396, 159859, 169208, 196971,
            198370, 200593, 206184, 210835, 212289, 215264, 215271, 216393,
            216570, 216749, 216776, 216805, 218521, 222605, 225099, 227162,
            247620, 253996, 257193, 258160, 268813, 276059, 276562],
           dtype='int64')


In [12]:
final_df=clean_df.drop(index_to_drop)

In [13]:
final_df['Flow Bytes per sec']=final_df['Flow Bytes per sec'].astype('float')
final_df['Flow Packets per sec']=final_df['Flow Packets per sec'].astype('float')

In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445645 entries, 0 to 445908
Data columns (total 79 columns):
Destination Port               445645 non-null int64
Flow Duration                  445645 non-null int64
Total Fwd Packets              445645 non-null int64
Total Backward Packets         445645 non-null int64
Total Length of Fwd Packets    445645 non-null int64
Total Length of Bwd Packets    445645 non-null int64
Fwd Packet Length Max          445645 non-null int64
Fwd Packet Length Min          445645 non-null int64
Fwd Packet Length Mean         445645 non-null float64
Fwd Packet Length Std          445645 non-null float64
Bwd Packet Length Max          445645 non-null int64
Bwd Packet Length Min          445645 non-null int64
Bwd Packet Length Mean         445645 non-null float64
Bwd Packet Length Std          445645 non-null float64
Flow Bytes per sec             445645 non-null float64
Flow Packets per sec           445645 non-null float64
Flow IAT Mean                

Finally, we notice there are two columns called **Fwd Header Length** and **Fwd Header Length.1**. We check if they are identical and drop one of them if true.

In [132]:
sum(final_df['Fwd Header Length']-final_df['Fwd Header Length.1'])

0

In [136]:
final_df.drop(['Fwd Header Length.1'],axis=1,inplace=True)

Now that our data frame is clean, we can pickle it into the cleaned data directory.

In [138]:
final_df.to_pickle('../data/cleaned/Tuesday_clean.pkl')

## Data Processing

Let's start by plotting histograms of the features we have in our dataset. All files are saved into the ./figures directory for later referencing.

In [139]:
features=final_df.columns.values

In [150]:
for index, feature in enumerate(features,1):
    fig = plt.figure()
    final_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/original_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

A cursory look at the histograms show us two things:

1. Some of the values seem to have a single spike indicating we need to look for features with a single value. Since these features carry no information, we should identify and remove them from the data frame.

2. The range of values for most of the attributes is quite large, resulting in largely skewed distributions. A log transform should be applied.

### 1.  Find the columns that have a single value and drop them from the dataframe

In [141]:
columns_to_drop=[]
for feature in features:
    if final_df[feature].unique().size==1:
        columns_to_drop.append(feature)
columns_to_drop

['Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'CWE Flag Count',
 'Fwd Avg Bytes per Bulk',
 'Fwd Avg Packets per Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes per Bulk',
 'Bwd Avg Packets per Bulk',
 'Bwd Avg Bulk Rate']

In [142]:
reduced_df=final_df.drop(columns_to_drop,axis=1)

In [143]:
reduced_features=reduced_df.columns.values
reduced_features

array(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes per sec',
       'Flow Packets per sec', 'Flow IAT Mean', 'Flow IAT Std',
       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets per sec', 'Bwd Packets per sec', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count',
       'RST Flag Count', 'PS

10 features have been removed, leaving us with 78-10=68 features. 

Let's save their histograms in a separate directory. This will help us choose features for further transformations to come from the remaining feature set.

In [149]:
for index, feature in enumerate(reduced_features,1):
    fig = plt.figure()
    reduced_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/reduced_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

### 2. Skewness transforms

The histograms also showed us that the range of values for some of the features was very large, resulting in largely skewed values. Now let's try to do a transform on the remaining features to try to reshape their distribution to a more "normal" looking form.

In [183]:
transform_candidates=np.array(['Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes per sec',
       'Flow Packets per sec', 'Flow IAT Mean', 'Flow IAT Std',
       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets per sec', 'Bwd Packets per sec', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance',  'Down Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size',
       'Avg Bwd Segment Size', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
       'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean',
       'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std',
       'Idle Max', 'Idle Min'])

There are 10 (one being the class label) features excluded from the transformation list. These features are binary valued, therefore transformation does not make sense.

In [199]:
no_transform_features=np.setdiff1d(reduced_features,transform_candidates)
no_transform_features

array(['ACK Flag Count', 'Destination Port', 'ECE Flag Count',
       'FIN Flag Count', 'Fwd PSH Flags', 'Label', 'PSH Flag Count',
       'RST Flag Count', 'SYN Flag Count', 'URG Flag Count'], dtype=object)

Since there are both positive and negative values across attributes as well as zero values, a log transform does not work. The standard transformer in this case is the 'Yeo-Johnson' transformation. 

Read more at: https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-transformer

In [249]:
transformed_df=pandas.DataFrame()

for feature in reduced_features:
    if feature in transform_candidates:
        transformed_df.loc[:,feature]=power_transform(reduced_df.loc[:,feature].values.reshape(-1,1),method='yeo-johnson')
    else:
        transformed_df.loc[:,feature]=reduced_df.loc[:,feature]

In [260]:
transformed_df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,445645.000000,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,...,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05,4.456450e+05
mean,8866.788354,-4.204151e-16,-1.478596e-15,4.946511e-16,3.346994e-16,-1.530638e-16,3.469445e-17,-2.908212e-17,4.591913e-18,3.571488e-17,...,1.377574e-17,6.790929e-16,4.591913e-17,5.459274e-17,7.142976e-18,-6.836848e-17,-1.392880e-16,-1.933706e-16,-2.239833e-16,5.408253e-17
std,19512.772737,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,0.000000,-4.903558e+00,-1.282462e+00,-1.776134e+00,-1.458646e+00,-1.288733e+00,-1.454952e+00,-9.092534e-01,-1.499533e+00,-6.762445e-01,...,-1.075478e+00,-1.317033e+01,-4.107866e-01,-3.097831e-01,-4.107809e-01,-4.106558e-01,-4.112639e-01,-3.104871e-01,-4.112639e-01,-4.112639e-01
25%,53.000000,-9.011295e-01,-1.900036e-01,-4.587906e-01,-7.721940e-01,-1.288733e+00,-1.190384e+00,-9.092534e-01,-1.251646e+00,-6.762445e-01,...,-1.075478e+00,-9.425081e-01,-4.107866e-01,-3.097831e-01,-4.107809e-01,-4.106558e-01,-4.112639e-01,-3.104871e-01,-4.112639e-01,-4.112639e-01
50%,80.000000,1.516240e-01,-1.900036e-01,1.199023e-01,1.215606e-01,2.461091e-01,1.135191e-01,-9.092534e-01,3.183552e-01,-6.762445e-01,...,2.815454e-01,-9.425081e-01,-4.107866e-01,-3.097831e-01,-4.107809e-01,-4.106558e-01,-4.112639e-01,-3.104871e-01,-4.112639e-01,-4.112639e-01
75%,443.000000,6.754302e-01,6.893957e-01,1.199023e-01,4.507421e-01,5.152729e-01,3.061337e-01,1.084638e+00,5.357581e-01,1.312998e+00,...,8.042686e-01,9.089398e-01,-4.107866e-01,-3.097831e-01,-4.107809e-01,-4.106558e-01,-4.112639e-01,-3.104871e-01,-4.112639e-01,-4.112639e-01
max,65505.000000,1.681029e+00,2.031974e+00,2.604905e+00,5.732889e+00,4.513705e+00,3.773028e+00,2.205676e+00,6.297168e+00,1.712164e+00,...,2.269216e+00,3.877064e+01,2.440292e+00,3.230956e+00,2.440923e+00,2.442968e+00,2.433451e+00,3.221524e+00,2.433441e+00,2.433515e+00


We save the histograms of the transformed features once again to confirm that transformations have worked as intended.

In [254]:
for index, feature in enumerate(reduced_features,1):
    fig = plt.figure()
    transformed_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/transformed_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

We finally pickle the resulting data frame after processing into the processed data directory.

In [255]:
transformed_df.to_pickle('../data/processed/Tuesday_processed.pkl')